# Experiment Data Aggregate Analysis

This notebook:
- Loads all Excel files in `data/` matching `experiment_data_*.xlsx`.
- Removes duplicate rows by `trialIndex` within each file and combines them.
- Groups by `experimentType` (from experiment data) to compute collaboration success rate.
- Loads and aggregates Questionnaire data, and plots answer distributions by RL type.

Notes:
- Expected sheets: `Experiment Data`, `Questionnaire Data`.
- Success column: `collaborationSucceeded` (boolean or 0/1).
- RL type column: `rlAgentType` (values like `individual`, `joint`).
- Columns may vary; this notebook handles missing columns gracefully.

In [1]:
# Imports & setup
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 180)
sns.set(style='whitegrid')

DATA_DIR = Path('data')
EXCEL_GLOB = 'experiment_data_*.xlsx'
SAVE_COMBINED_CSV = True
COMBINED_EXPERIMENT_CSV = DATA_DIR / 'combined_experiment_data.csv'
COMBINED_QUESTIONNAIRE_CSV = DATA_DIR / 'combined_questionnaire_data.csv'

excel_paths = sorted([p for p in DATA_DIR.glob(EXCEL_GLOB) if not p.name.startswith('~$')])
len(excel_paths), excel_paths[:3]

(0, [])

In [2]:
# Helper functions
def file_id_from_path(p: Path) -> str:
    return p.stem

def load_experiment_data_one_file(p: Path) -> pd.DataFrame:
    """Read 'Experiment Data' sheet, add file columns, drop duplicates by trialIndex.
    Returns empty DataFrame if sheet missing.
    """
    try:
        xls = pd.ExcelFile(p)
    except Exception as e:
        print(f'[WARN] Failed to open {p.name}: {e}')
        return pd.DataFrame()

    sheet = None
    for candidate in ['Experiment Data', 'experiment data', 'ExperimentData', 'Data']:
        if candidate in xls.sheet_names:
            sheet = candidate
            break
    if sheet is None:
        print(f'[INFO] No Experiment Data sheet in {p.name}. Found: {xls.sheet_names}')
        return pd.DataFrame()

    try:
        df = pd.read_excel(p, sheet_name=sheet)
    except Exception as e:
        print(f'[WARN] Failed to read {sheet} from {p.name}: {e}')
        return pd.DataFrame()

    # Normalize columns
    if 'trialIndex' not in df.columns:
        # Try common variants
        for alt in ['trial_index', 'trial', 'trialId']:
            if alt in df.columns:
                df = df.rename(columns={alt: 'trialIndex'})
                break
    if 'experimentType' not in df.columns:
        for alt in ['experiment_type', 'experiment']:
            if alt in df.columns:
                df = df.rename(columns={alt: 'experimentType'})
                break
    if 'collaborationSucceeded' not in df.columns:
        for alt in ['collaboration_success', 'success', 'collabSuccess']:
            if alt in df.columns:
                df = df.rename(columns={alt: 'collaborationSucceeded'})
                break

    # Drop duplicate trials within file
    if 'trialIndex' in df.columns:
        before = len(df)
        df = df.drop_duplicates(subset=['trialIndex'], keep='last')
        after = len(df)
        if after < before:
            print(f'[INFO] {p.name}: dropped {before - after} duplicate trials by trialIndex')
    else:
        print(f'[INFO] {p.name}: missing trialIndex; no de-duplication applied')

    df['source_file'] = p.name
    df['file_id'] = file_id_from_path(p)
    return df

def load_questionnaire_one_file(p: Path) -> pd.DataFrame:
    """Read 'Questionnaire Data' if present and add file columns.
    Returns empty DataFrame if sheet missing.
    """
    try:
        xls = pd.ExcelFile(p)
    except Exception as e:
        print(f'[WARN] Failed to open {p.name}: {e}')
        return pd.DataFrame()

    qsheet = None
    for candidate in ['Questionnaire Data', 'questionnaire data', 'Questionnaire', 'Survey']:
        if candidate in xls.sheet_names:
            qsheet = candidate
            break
    if qsheet is None:
        # Not all files have questionnaire
        return pd.DataFrame()

    try:
        qdf = pd.read_excel(p, sheet_name=qsheet)
    except Exception as e:
        print(f'[WARN] Failed to read {qsheet} from {p.name}: {e}')
        return pd.DataFrame()

    qdf['source_file'] = p.name
    qdf['file_id'] = file_id_from_path(p)
    return qdf

def mode_or_first(series: pd.Series):
    if series is None or series.empty:
        return np.nan
    try:
        m = series.mode(dropna=True)
        return m.iloc[0] if not m.empty else series.dropna().iloc[0] if series.dropna().shape[0] else np.nan
    except Exception:
        return series.dropna().iloc[0] if series.dropna().shape[0] else np.nan

def bool_to_int(s: pd.Series) -> pd.Series:
    if s.dtype == bool:
        return s.astype(int)
    # try to coerce truthy strings
    return s.replace({'true': 1, 'false': 0, 'True': 1, 'False': 0}).astype(float, errors='ignore')


In [3]:
# Load and combine experiment data
exp_frames = []
for p in excel_paths:
    df = load_experiment_data_one_file(p)
    if not df.empty:
        exp_frames.append(df)

combined_exp = pd.concat(exp_frames, ignore_index=True) if exp_frames else pd.DataFrame()
print('Combined experiment rows:', len(combined_exp))
if not combined_exp.empty:
    # Normalize types
    if 'collaborationSucceeded' in combined_exp.columns:
        combined_exp['collaborationSucceeded'] = bool_to_int(combined_exp['collaborationSucceeded']).astype(float)
    if 'trialIndex' in combined_exp.columns:
        combined_exp['trialIndex'] = pd.to_numeric(combined_exp['trialIndex'], errors='coerce')
    if SAVE_COMBINED_CSV:
        combined_exp.to_csv(COMBINED_EXPERIMENT_CSV, index=False)
        print(f'Saved combined experiment data -> {COMBINED_EXPERIMENT_CSV}')
combined_exp.head(3)

Combined experiment rows: 0


""


In [4]:
# Success rate by experimentType
if combined_exp.empty:
    print('No experiment data found.')
else:
    needed = {'experimentType', 'collaborationSucceeded'}
    if not needed.issubset(set(combined_exp.columns)):
        print(f'Missing columns for success rate: {needed - set(combined_exp.columns)}')
    else:
        grp = combined_exp.groupby('experimentType')['collaborationSucceeded'].agg(['count','sum','mean']).reset_index()
        grp['success_rate_percent'] = (grp['mean'] * 100).round(1)
        display(grp.sort_values('success_rate_percent', ascending=False))

        plt.figure(figsize=(7,4))
        order = grp.sort_values('success_rate_percent', ascending=False)['experimentType']
        sns.barplot(data=grp, x='experimentType', y='success_rate_percent', order=order, color='#4C78A8')
        plt.ylabel('Success Rate (%)')
        plt.xlabel('Experiment Type')
        plt.title('Collaboration Success Rate by Experiment Type')
        for i, v in enumerate(grp.set_index('experimentType').loc[order]['success_rate_percent']):
            plt.text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom', fontsize=9)
        plt.tight_layout()
        plt.show()


No experiment data found.


In [5]:
# Load and combine questionnaire data
q_frames = []
for p in excel_paths:
    qdf = load_questionnaire_one_file(p)
    if not qdf.empty:
        q_frames.append(qdf)

combined_q = pd.concat(q_frames, ignore_index=True) if q_frames else pd.DataFrame()
print('Combined questionnaire rows:', len(combined_q))
if not combined_q.empty and SAVE_COMBINED_CSV:
    combined_q.to_csv(COMBINED_QUESTIONNAIRE_CSV, index=False)
    print(f'Saved combined questionnaire data -> {COMBINED_QUESTIONNAIRE_CSV}')
combined_q.head(3)

Combined questionnaire rows: 0


""


In [6]:
# Merge questionnaire with per-file RL type and experimentType from experiment data
if combined_q.empty or combined_exp.empty:
    print('Skip merge: missing questionnaire or experiment data')
else:
    per_file = combined_exp.groupby('file_id').agg({
        'rlAgentType': mode_or_first,
        'experimentType': mode_or_first
    }).reset_index().rename(columns={'rlAgentType': 'rl_type'})

    q_merged = combined_q.merge(per_file, on='file_id', how='left')
    print('Questionnaire merged shape:', q_merged.shape)
    display(q_merged.head(3))

    # Cache for later cells
    questionnaire_merged = q_merged.copy()


Skip merge: missing questionnaire or experiment data


In [7]:
# Plot: Play-again distribution by RL type (if available)
if 'questionnaire_merged' not in globals():
    print('No merged questionnaire to plot.')
else:
    q = questionnaire_merged.copy()
    # Try to find a 'play again' like column
    play_cols = [c for c in q.columns if c.lower().replace(' ', '').startswith('playagain')]
    if not play_cols:
        # common variants
        for cand in ['play_again', 'Would you play again?']:
            if cand in q.columns:
                play_cols = [cand]
                break
    if not play_cols:
        print('No play-again column found in questionnaire. Available columns example:', list(q.columns)[:20])
    else:
        col = play_cols[0]
        print('Using play-again column:', col)
        # Keep order if Likert-like values exist
        order = [
            'Definitely play again', 'Probably play again',
            'Might or might not', 'Probably not', 'Definitely not'
        ]
        # Aggregate counts
        agg = q.groupby(['rl_type', col], observed=True).size().reset_index(name='count')
        plt.figure(figsize=(8,4))
        sns.barplot(data=agg, x=col, y='count', hue='rl_type', hue_order=['individual','joint'] if 'individual' in agg['rl_type'].unique() else None, order=order if set(order).intersection(set(agg[col].astype(str))) else None)
        plt.xticks(rotation=20, ha='right')
        plt.title('Play-again responses by RL type')
        plt.xlabel('Response')
        plt.ylabel('Count')
        plt.legend(title='RL Type')
        plt.tight_layout()
        plt.show()


No merged questionnaire to plot.


In [8]:
# Optional: AI detection and collaboration rating plots if present
if 'questionnaire_merged' in globals():
    q = questionnaire_merged.copy()
    for target_col, title in [
        ('ai_detection', 'AI-likeness by RL type'),
        ('collaboration_rating', 'Collaboration rating by RL type')
    ]:
        if target_col in q.columns:
            agg = q.groupby(['rl_type', target_col], observed=True).size().reset_index(name='count')
            plt.figure(figsize=(8,4))
            sns.barplot(data=agg, x=target_col, y='count', hue='rl_type', hue_order=['individual','joint'] if 'individual' in agg['rl_type'].unique() else None)
            plt.xticks(rotation=20, ha='right')
            plt.title(title)
            plt.xlabel('Response')
            plt.ylabel('Count')
            plt.legend(title='RL Type')
            plt.tight_layout()
            plt.show()
        else:
            print(f'Skip plot: {target_col} not found in questionnaire columns')
